In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/RGB_Li_192c_J4_nf8_v1.0.2.pth

In [1]:
import io
import time
import torch
import numpy as np
import PIL
from torchvision.transforms import ToPILImage, PILToTensor
from datasets import load_dataset, Image
from walloc import walloc

class Config: pass

In [2]:
checkpoint = torch.load("RGB_Li_192c_J4_nf8_v1.0.2.pth",map_location="cpu",weights_only=False)
codec_config = checkpoint['config']
codec = walloc.Codec2D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode
)
codec.load_state_dict(checkpoint['model_state_dict'])
del codec.decoder
codec.eval();

In [3]:
dataset = load_dataset("danjacobellis/LSDIR_val", split='validation')

In [4]:
def walloc_compress(sample):
    with torch.no_grad():
        img = sample['image']
        x = PILToTensor()(img).to(torch.float)
        x = (x/255 - 0.5).unsqueeze(0).to(device)
        H, W = x.size(2), x.size(3)
        x_padded = walloc.pad(x,p=8)

        t0 = time.time()
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        webp = walloc.latent_to_pil(Y.to("cpu"),codec.latent_bits, 3)[0]
        buff = io.BytesIO()
        webp.save(buff, format='WEBP', lossless=True)
        webp_bytes = buff.getbuffer()
        encode_time = time.time() - t0
        
        bpp = 8*len(webp_bytes)/(H*W)  
        
    return {
        'compressed': webp_bytes,
        'encode_time': encode_time,
        'bpp': bpp,
    }

In [5]:
device = "cpu"
codec = codec.to(device)
walloc_ds = dataset.map(walloc_compress, writer_batch_size=10)
walloc_ds = walloc_ds.cast_column('compressed',Image())

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [9]:
np.median(walloc_ds['encode_time'])

1.3567014932632446